In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy.io.wavfile as wav
import sys
import sounddevice as sd

# Add the current directory to the path so we can import our modules
sys.path.append(".")

from plot import plot_continuous, plot_continuous_discrete
from frames import non_overlapping_frames, overlapping_frames
from fundamental_frecuency import calculate_fundamental_frequency
from collections import namedtuple

## Ejercicio 1.1
Escuchamos las señales y las representamos en el tiempo continuo y en discreto.

In [ ]:
AUDIO_DIR = "../audio"

# set numpy print options to print only 10 elements of an array
np.set_printoptions(threshold=10)

# Create a named tuple to represent a signal
Signal = namedtuple("Signal", ["y", "fs", "file"])

# Read all the .wav files in the audio directory
signals = []
for file in sorted(os.listdir(AUDIO_DIR)):
    if file.endswith(".wav"):
        # get the signal and the sampling frequency
        fs, y = wav.read(os.path.join(AUDIO_DIR, file))

        # normalize the signal to match matlab behavior
        # signals are 16 bit signed integers
        y = y / 2**15

        # create a Signal object and append it to the list
        signals.append(Signal(y, fs, file))

# Plot and play the signals
for signal in signals:
    plot_continuous_discrete(signal.y, signal.fs, title=signal.file)

    # Play the audio
    # sd.play(signal.y, signal.fs)
    # sd.wait()

## Ejercicio 1.2
Representamos un tramo de 100ms de cada señal en tiempo continuo y calculamos la potencia media, la energía y el periodo fundamental de las señales que sean periodicas.

In [ ]:
# values chosen by hand to get the relevant part of the signals
# frames_start = [15000, 14000, 1000, 18000, 12300]
frames_start = {
    "sound1.wav": 15000,
    "sound2.wav": 14000,
    "sound3.wav": 1000,
    "sound4.wav": 18000,
    "sound5.wav": 12300,
}

for signal in signals:
    # calculate the length of the frames
    frame_duration = 0.1
    yframe_len = round(frame_duration * signal.fs)

    # extract the frames from the original signals.
    yframe = signal.y[
        frames_start[signal.file] : frames_start[signal.file] + yframe_len
    ]

    # plot the frames
    plot_continuous(yframe, signal.fs, title=f"{signal.file} frame")

    # calculate the fundamental frequency of the signals
    fundamental_frequency = calculate_fundamental_frequency(yframe, signal.fs)
    print(f"Fundamental frecuecny of signal {signal.file}: {fundamental_frequency} hz")

    # calculate the power and energy of the signals
    print(f"Power of signal {signal.file}: {(signal.y ** 2).mean()} W")
    print(f"Energy of signal {signal.file}: {(signal.y ** 2).sum()} J")

## Ejercicio 1.3
Calculamos la energía de las señales en tramas de 100ms sin solape y las representamos.

In [ ]:
E_frames = []

for signal in signals:
    # get the energy of the signals in overlapping frames of 100ms
    frame_duration = 0.1
    E_frame = (non_overlapping_frames(signal.y, signal.fs, frame_duration) ** 2).sum(
        axis=0
    )
    E_frames.append(E_frame)

    # plot the energy of the signals in continuous time
    fs = 1 / frame_duration
    plot_continuous(E_frame, fs, title=f"{signal.file} energy", ylabel="Energy (J)")
    print(f"Energy of signal {signal.file} (non overlapping frames): {E_frame} J")

for E_frame in E_frames:
    # add the energy of the signal to the final plot with the energy of all the signals
    t = np.arange(len(E_frame)) * signal.fs
    plt.plot(t, E_frame)

# plot the energy of all the signals in discrete time
plt.xlabel("n")
plt.ylabel("Energy (J)")
plt.title("Energy of the signals (non overlapping frames)")
plt.legend([signal.file for signal in signals])
plt.show()

## Ejercicio 1.4
Calculamos la energía de las señales en tramas de 20ms con solape y las representamos.

In [ ]:
E_frames = []

for signal in signals:
    # get the energy of the signals in overlapping frames of 100ms
    overlap = 0.5
    frame_duration = 0.02
    E_frame = (
        overlapping_frames(signal.y, signal.fs, frame_duration, overlap) ** 2
    ).sum(axis=0)
    E_frames.append(E_frame)

    # plot the energy of the signals in continuous time
    fs = 1 / (frame_duration * overlap)
    plot_continuous(E_frame, fs, title=f"{signal.file} energy", ylabel="Energy (J)")
    print(f"Energy of signal {signal.file} (non overlapping frames): {E_frame} J")

for E_frame in E_frames:
    # add the energy of the signal to the final plot with the energy of all the signals
    t = np.arange(len(E_frame)) * signal.fs
    plt.plot(t, E_frame)

# plot the energy of all the signals in discrete time
plt.xlabel("n")
plt.ylabel("Energy (J)")
plt.title("Energy of the signals (non overlapping frames)")
plt.legend([signal.file for signal in signals])
plt.show()